# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Class-that-matches-and-combines-boxes" data-toc-modified-id="Class-that-matches-and-combines-boxes-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Class that matches and combines boxes</a></div><div class="lev2 toc-item"><a href="#Class-encapsulating-data-about-boxes" data-toc-modified-id="Class-encapsulating-data-about-boxes-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Class encapsulating data about boxes</a></div><div class="lev2 toc-item"><a href="#Parse-the-downloaded-subject-data-into-data-structures-for-processing" data-toc-modified-id="Parse-the-downloaded-subject-data-into-data-structures-for-processing-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Parse the downloaded subject data into data structures for processing</a></div><div class="lev2 toc-item"><a href="#Parse-the-downloaded-box-data-into-data-structures-for-processing" data-toc-modified-id="Parse-the-downloaded-box-data-into-data-structures-for-processing-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Parse the downloaded box data into data structures for processing</a></div><div class="lev2 toc-item"><a href="#Identify-likely-matches-between-classifier-box-definitions" data-toc-modified-id="Identify-likely-matches-between-classifier-box-definitions-05"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Identify likely matches between classifier box definitions</a></div><div class="lev2 toc-item"><a href="#Add-subject-data-to-aggregated-box-details" data-toc-modified-id="Add-subject-data-to-aggregated-box-details-06"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>Add subject data to aggregated box details</a></div><div class="lev2 toc-item"><a href="#Quick-test:-Plot-all-boxes-for-the-first-subject-key" data-toc-modified-id="Quick-test:-Plot-all-boxes-for-the-first-subject-key-07"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>Quick test: Plot all boxes for the first subject key</a></div><div class="lev2 toc-item"><a href="#Write-aggregated-box-data-to-MySQL-database" data-toc-modified-id="Write-aggregated-box-data-to-MySQL-database-08"><span class="toc-item-num">0.8&nbsp;&nbsp;</span>Write aggregated box data to MySQL database</a></div>

In [1]:
import json
import csv
import pandas as pd
import numpy as np
from PIL import Image
import urllib
import matplotlib
import matplotlib.pyplot as mplplot
import matplotlib.patches as patches
import dateutil.parser
import pickle
import sys
import regex
import copy
import itertools
import gc
from collections import Counter

In [2]:
verbose = True
if verbose:
    import pprint
    from IPython.core.display import display
    pprinter = pprint.PrettyPrinter(indent=4)

In [3]:
#%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'jpeg')

In [4]:
sampleDataFileName = 'classification_export_mssEC_03_1_19_17.csv'  #'decoding-the-civil-war-classifications.csv'
subjectDataFileName = 'decoding-the-civil-war-subjects.csv'
databaseName = 'dcwConsensus_mssEC_03_1_19_17'
liveDate = dateutil.parser.parse("2016-06-20T00:00:00.00Z")

## Class that matches and combines boxes

In [5]:
class BoxMatcher():
    def __init__(self, _overlapThreshold=0.7):
        self.overlapThreshold = _overlapThreshold
        self.box = None

    def compare(self, otherBox):
        if self.box is None:
            self.setBox(otherBox)
            return True
        # define "identity" as a degree of area overlap
        selfArea = self.box.width * self.box.height
        otherArea = otherBox.width * otherBox.height
        dx = min(self.box.x + self.box.width,
                 otherBox.x + otherBox.width) - max(self.box.x, otherBox.x)
        dy = min(self.box.y + self.box.height,
                 otherBox.y + otherBox.height) - max(self.box.y, otherBox.y)
        if dx < 0 or dy < 0:
            return False
        areaOfOverlap = dx * dy
        unionOfAreas = selfArea + otherArea - areaOfOverlap
        overlapFraction = areaOfOverlap / unionOfAreas
        return overlapFraction > self.overlapThreshold

    def setBox(self, newBox):
        self.box = newBox

    @staticmethod
    def mean(boxes):

        meanCoMX = meanCoMY = meanWidth = meanHeight = 0.0
        for box in boxes:
            boxCoM = (box.x + 0.5 * box.width, box.y + 0.5 * box.height)
            meanCoMX += boxCoM[0]
            meanCoMY += boxCoM[1]
            meanWidth += box.width
            meanHeight += box.height

        meanBox = TelegramBox(
            meanCoMX / float(len(boxes)) - 0.5 * meanWidth / float(len(boxes)),
            meanCoMY / float(len(boxes)) -
            0.5 * meanHeight / float(len(boxes)),
            meanWidth / float(len(boxes)), meanHeight / float(len(boxes)), {
                'nBoxes': len(boxes)
            })
        return meanBox

## Class encapsulating data about boxes

In [6]:
class TelegramBox():
    def __init__(self, x, y, width, height, data):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.data = data

    def __str__(self):
        return 'TelegramBox(x={}, y={}, width={}, height={}, subject={})'.format(
            self.x, self.y, self.width, self.height, self.data)

    def __repr__(self):
        return self.__str__()

## Parse the downloaded subject data into data structures for processing

In [7]:
subject_data = []
subjectColumns = ['subject_id', 'huntington_id', 'url']
with open(subjectDataFileName) as csvfile:
    parsedSubjectCsv = csv.DictReader(csvfile)
    numPrinted = 0
    for subject in parsedSubjectCsv:
        parsedLocations = json.loads(subject['locations'])
        parsedMetaData = json.loads(subject['metadata'])
        if 'hdl_id' not in parsedMetaData:
            continue
        subject_data.append({
            'subject_id':
            int(subject['subject_id']),
            'huntington_id':
            parsedMetaData['hdl_id'],
            'collection':
            parsedMetaData['hdl_id'][3:5],
            'ledger':
            int(parsedMetaData['hdl_id'][6:8]),
            'page':
            int(parsedMetaData['hdl_id'][9:12]),
            'variant':
            parsedMetaData['hdl_id'][12:],
            'telegramNumbers': [
                int(regex.findall(r'[0-9]+', telegramNumber)[0])
                for telegramNumber in regex.findall(r"[\w']+", parsedMetaData[
                    '#telegrams']) if len(telegramNumber) > 3
            ],
            'url':
            parsedLocations['0']
        })
subjectsFrame = pd.DataFrame.from_records(subject_data, index='subject_id')
display(subjectsFrame)

,collection,huntington_id,ledger,page,telegramNumbers,url,variant
subject_id,,,,,,,
1959268,EC,mssEC_01_001,1,1,[],https://panoptes-uploads.zooniverse.org/produc...,
1959269,EC,mssEC_01_002,1,2,[],https://panoptes-uploads.zooniverse.org/produc...,
1959270,EC,mssEC_01_003,1,3,[],https://panoptes-uploads.zooniverse.org/produc...,
1959271,EC,mssEC_01_004,1,4,[],https://panoptes-uploads.zooniverse.org/produc...,
1959272,EC,mssEC_01_005,1,5,[],https://panoptes-uploads.zooniverse.org/produc...,
1959273,EC,mssEC_01_006,1,6,[],https://panoptes-uploads.zooniverse.org/produc...,
1959274,EC,mssEC_01_007,1,7,"[1, 2, 3]",https://panoptes-uploads.zooniverse.org/produc...,
1959275,EC,mssEC_01_008,1,8,[4],https://panoptes-uploads.zooniverse.org/produc...,
1959276,EC,mssEC_01_009,1,9,[5],https://panoptes-uploads.zooniverse.org/produc...,


## Parse the downloaded box data into data structures for processing

In [8]:
allBoxes = {}

onePrinted = False

with open(sampleDataFileName) as csvfile:
    parsedCsv = csv.DictReader(csvfile)
    nSubjectsParsed = 0
    for recordIndex, record in enumerate(parsedCsv):

        subjectBoxes = []

        # check the date that the classification was made
        if 'metadata' in record:
            parsedMetadata = json.loads(record["metadata"])
            parsedDate = dateutil.parser.parse(parsedMetadata['started_at'])
            # skip "testing" data before the site went live
            if parsedDate < liveDate:
                continue

        # parse the annotations and the subject data
        parsedAnnotations = json.loads(record["annotations"])
        parsedSubjectData = json.loads(record["subject_data"])

        #loop over tasks in the annotation
        for task in parsedAnnotations:
            # Check for recorded box data
            if task['task'] == "T2" and task['value']:
                for box in task['value']:
                    subjectBoxes.append(
                        TelegramBox(box['x'], box['y'], box['width'], box[
                            'height'], int(record['subject_ids'])))

        nSubjectsParsed += 1
        if int(record['subject_ids']) in allBoxes:
            allBoxes[int(record['subject_ids'])].append((recordIndex,
                                                         subjectBoxes))
        else:
            allBoxes.update({
                int(record['subject_ids']): [(recordIndex, subjectBoxes)]
            })

In [9]:
allBoxData = []
for key, boxes in allBoxes.items():
    for boxData in boxes:
        for boxDatum in boxData[1]:
            allBoxData.append({
                'subjectKey': key,
                'box': boxDatum,
                'boxX': boxDatum.x,
                'boxY': boxDatum.y,
                'boxW': boxDatum.width,
                'boxH': boxDatum.height
            })

allBoxesFrame = pd.DataFrame(data=allBoxData)

allBoxesFrameIndex = pd.MultiIndex.from_arrays([
    allBoxesFrame['subjectKey'],
    allBoxesFrame['boxY'],
    allBoxesFrame['boxH'],
    allBoxesFrame['boxX'],
    allBoxesFrame['boxW'],
])

allBoxesFrame.set_index(allBoxesFrameIndex, inplace=True, drop=False)
allBoxesFrame.sort_index(inplace=True)
display(allBoxesFrame)

box  \
subjectKey boxY        boxH        boxX       boxW                                                             
1959735    69.208716   1571.544248 183.670132 1119.155508  TelegramBox(x=183.67013160916903, y=69.2087155...   
           77.498967   634.546645  217.872554 1064.254884  TelegramBox(x=217.87255407222628, y=77.4989665...   
           88.380022   619.367235  188.993039 1122.072843  TelegramBox(x=188.9930393607017, y=88.38002242...   
           90.818091   652.755032  256.841798 1021.703532  TelegramBox(x=256.8417977185539, y=90.81809142...   
           94.137471   618.447434  226.844412 1053.386382  TelegramBox(x=226.84441180954198, y=94.1374706...   
           96.377308   619.568408  201.214265 1096.280664  TelegramBox(x=201.21426458586689, y=96.3773079...   
           97.430817   610.066810  227.723913 1055.250813  TelegramBox(x=227.72391260977312, y=97.4308173...   
           726.983791  869.382678  205.239875 1080.732722  TelegramBox(x=205.23987539791085, y=726.983790...   
           727.992879  867.395771  206.377282 1084.233624  TelegramBox(x=206.37728184172266, y=727.992879...   
           731.071666  874.172775  219.929085 1063.226618  TelegramBox(x=219.92908524905818, y=731.071665...   
           731.079585  918.445249  169.902953 1158.131895  TelegramBox(x=169.90295318326912, y=731.079584...   
           731.650746  894.901778  208.970208 1085.559948  TelegramBox(x=208.9702084974614, y=731.6507456...   
           739.316026  859.933803  210.013719 1077.045807  TelegramBox(x=210.01371916363914, y=739.316025...   
1959736    60.410355   1228.919217 189.855783 1113.276583  TelegramBox(x=189.85578296643678, y=60.4103547...   
           78.869204   1167.264212 197.877674 1065.340910  TelegramBox(x=197.8776744205451, y=78.86920351...   
           81.775750   1131.522772 211.351414 1353.891507  TelegramBox(x=211.35141431929688, y=81.7757503...   
           88.160784   1179.180621 205.201345 1110.326714  TelegramBox(x=205.20134545513017, y=88.1607841...   
           114.739998  1142.810477 207.720285 1107.405005  TelegramBox(x=207.72028545098922, y=114.739997...   
           139.368102  1094.724281 187.253129 1087.205472  TelegramBox(x=187.25312884161386, y=139.368101...   
           142.430851  1085.883071 192.374103 1085.883167  TelegramBox(x=192.37410295649133, y=142.430851...   
           155.074797  1153.050819 202.534998 1094.795611  TelegramBox(x=202.53499796962183, y=155.074796...   
1959737    78.495796   1786.456159 257.023403 1045.106835  TelegramBox(x=257.0234028108319, y=78.49579634...   
           80.953856   1785.433153 274.882492 1038.166705  TelegramBox(x=274.8824923424859, y=80.95385632...   
           81.635935   1931.857936 287.191704 1025.096856  TelegramBox(x=287.19170354908823, y=81.6359351...   
           83.940396   1795.724910 205.239875 1100.218888  TelegramBox(x=205.23987539791085, y=83.9403964...   
           87.568574   1755.900889 245.980583 1058.371989  TelegramBox(x=245.98058344712456, y=87.5685739...   
           91.438641   1773.675171 253.096775 1069.125824  TelegramBox(x=253.0967750275375, y=91.43864064...   
           94.137471   1762.396445 255.443137 1052.194768  TelegramBox(x=255.44313710887087, y=94.1374706...   
           98.829675   1788.135046 229.936208 1081.097349  TelegramBox(x=229.93620779036635, y=98.8296752...   
           100.394599  1854.347305 234.185840 1112.214702  TelegramBox(x=234.18584038154205, y=100.394599...   
...                                                                                                      ...   
1959791    86.905645   1867.127471 191.776824 1101.999462  TelegramBox(x=191.77682415418474, y=86.9056452...   
           87.861369   1864.928409 197.264597 1092.126256  TelegramBox(x=197.2645966816108, y=87.86136880...   
           98.178411   1858.447895 213.427712 1076.987417  TelegramBox(x=213.4277122678626, y=98.17841065...   
           102.500000  1848.000000 246.468750 1039.000000  TelegramBox(x=

## Identify likely matches between classifier box definitions

In [10]:
# Add new column listing the most likely box id
allBoxesFrame['bestBoxIndex'] = pd.Series(
    np.zeros_like(allBoxesFrame['subjectKey']), index=allBoxesFrame.index)
subjectKey = None
groupIndex = None
boxMatcher = BoxMatcher(0.7)

for index, data in allBoxesFrame.iterrows():
    if subjectKey != index[0]:
        subjectKey = index[0]
        groupIndex = 0
    if not boxMatcher.compare(data['box']):
        groupIndex += 1
    allBoxesFrame.ix[index, 'bestBoxIndex'] = groupIndex
    boxMatcher.setBox(data['box'])

display(allBoxesFrame[['box', 'bestBoxIndex']])

box  \
subjectKey boxY        boxH        boxX       boxW                                                             
1959735    69.208716   1571.544248 183.670132 1119.155508  TelegramBox(x=183.67013160916903, y=69.2087155...   
           77.498967   634.546645  217.872554 1064.254884  TelegramBox(x=217.87255407222628, y=77.4989665...   
           88.380022   619.367235  188.993039 1122.072843  TelegramBox(x=188.9930393607017, y=88.38002242...   
           90.818091   652.755032  256.841798 1021.703532  TelegramBox(x=256.8417977185539, y=90.81809142...   
           94.137471   618.447434  226.844412 1053.386382  TelegramBox(x=226.84441180954198, y=94.1374706...   
           96.377308   619.568408  201.214265 1096.280664  TelegramBox(x=201.21426458586689, y=96.3773079...   
           97.430817   610.066810  227.723913 1055.250813  TelegramBox(x=227.72391260977312, y=97.4308173...   
           726.983791  869.382678  205.239875 1080.732722  TelegramBox(x=205.23987539791085, y=726.983790...   
           727.992879  867.395771  206.377282 1084.233624  TelegramBox(x=206.37728184172266, y=727.992879...   
           731.071666  874.172775  219.929085 1063.226618  TelegramBox(x=219.92908524905818, y=731.071665...   
           731.079585  918.445249  169.902953 1158.131895  TelegramBox(x=169.90295318326912, y=731.079584...   
           731.650746  894.901778  208.970208 1085.559948  TelegramBox(x=208.9702084974614, y=731.6507456...   
           739.316026  859.933803  210.013719 1077.045807  TelegramBox(x=210.01371916363914, y=739.316025...   
1959736    60.410355   1228.919217 189.855783 1113.276583  TelegramBox(x=189.85578296643678, y=60.4103547...   
           78.869204   1167.264212 197.877674 1065.340910  TelegramBox(x=197.8776744205451, y=78.86920351...   
           81.775750   1131.522772 211.351414 1353.891507  TelegramBox(x=211.35141431929688, y=81.7757503...   
           88.160784   1179.180621 205.201345 1110.326714  TelegramBox(x=205.20134545513017, y=88.1607841...   
           114.739998  1142.810477 207.720285 1107.405005  TelegramBox(x=207.72028545098922, y=114.739997...   
           139.368102  1094.724281 187.253129 1087.205472  TelegramBox(x=187.25312884161386, y=139.368101...   
           142.430851  1085.883071 192.374103 1085.883167  TelegramBox(x=192.37410295649133, y=142.430851...   
           155.074797  1153.050819 202.534998 1094.795611  TelegramBox(x=202.53499796962183, y=155.074796...   
1959737    78.495796   1786.456159 257.023403 1045.106835  TelegramBox(x=257.0234028108319, y=78.49579634...   
           80.953856   1785.433153 274.882492 1038.166705  TelegramBox(x=274.8824923424859, y=80.95385632...   
           81.635935   1931.857936 287.191704 1025.096856  TelegramBox(x=287.19170354908823, y=81.6359351...   
           83.940396   1795.724910 205.239875 1100.218888  TelegramBox(x=205.23987539791085, y=83.9403964...   
           87.568574   1755.900889 245.980583 1058.371989  TelegramBox(x=245.98058344712456, y=87.5685739...   
           91.438641   1773.675171 253.096775 1069.125824  TelegramBox(x=253.0967750275375, y=91.43864064...   
           94.137471   1762.396445 255.443137 1052.194768  TelegramBox(x=255.44313710887087, y=94.1374706...   
           98.829675   1788.135046 229.936208 1081.097349  TelegramBox(x=229.93620779036635, y=98.8296752...   
           100.394599  1854.347305 234.185840 1112.214702  TelegramBox(x=234.18584038154205, y=100.394599...   
...                                                                                                      ...   
1959791    86.905645   1867.127471 191.776824 1101.999462  TelegramBox(x=191.77682415418474, y=86.9056452...   
           87.861369   1864.928409 197.264597 1092.126256  TelegramBox(x=197.2645966816108, y=87.86136880...   
           98.178411   1858.447895 213.427712 1076.987417  TelegramBox(x=213.4277122678626, y=98.17841065...   
           102.500000  1848.000000 246.468750 1039.000000  TelegramBox(x=

In [11]:
allBoxesFrameReindexed = allBoxesFrame.reset_index(
    level=[1, 2, 3, 4], drop=True)
allBoxesFrameReindexed.set_index('bestBoxIndex', append=True, inplace=True)
display(allBoxesFrameReindexed)

box  \
subjectKey bestBoxIndex                                                      
1959735    0             TelegramBox(x=183.67013160916903, y=69.2087155...   
           1             TelegramBox(x=217.87255407222628, y=77.4989665...   
           1             TelegramBox(x=188.9930393607017, y=88.38002242...   
           1             TelegramBox(x=256.8417977185539, y=90.81809142...   
           1             TelegramBox(x=226.84441180954198, y=94.1374706...   
           1             TelegramBox(x=201.21426458586689, y=96.3773079...   
           1             TelegramBox(x=227.72391260977312, y=97.4308173...   
           2             TelegramBox(x=205.23987539791085, y=726.983790...   
           2             TelegramBox(x=206.37728184172266, y=727.992879...   
           2             TelegramBox(x=219.92908524905818, y=731.071665...   
           2             TelegramBox(x=169.90295318326912, y=731.079584...   
           2             TelegramBox(x=208.9702084974614, y=731.6507456...   
           2             TelegramBox(x=210.01371916363914, y=739.316025...   
1959736    1             TelegramBox(x=189.85578296643678, y=60.4103547...   
           1             TelegramBox(x=197.8776744205451, y=78.86920351...   
           1             TelegramBox(x=211.35141431929688, y=81.7757503...   
           1             TelegramBox(x=205.20134545513017, y=88.1607841...   
           1             TelegramBox(x=207.72028545098922, y=114.739997...   
           1             TelegramBox(x=187.25312884161386, y=139.368101...   
           1             TelegramBox(x=192.37410295649133, y=142.430851...   
           1             TelegramBox(x=202.53499796962183, y=155.074796...   
1959737    1             TelegramBox(x=257.0234028108319, y=78.49579634...   
           1             TelegramBox(x=274.8824923424859, y=80.95385632...   
           1             TelegramBox(x=287.19170354908823, y=81.6359351...   
           1             TelegramBox(x=205.23987539791085, y=83.9403964...   
           1             TelegramBox(x=245.98058344712456, y=87.5685739...   
           1             TelegramBox(x=253.0967750275375, y=91.43864064...   
           1             TelegramBox(x=255.44313710887087, y=94.1374706...   
           1             TelegramBox(x=229.93620779036635, y=98.8296752...   
           1             TelegramBox(x=234.18584038154205, y=100.394599...   
...                                                                    ...   
1959791    1             TelegramBox(x=191.77682415418474, y=86.9056452...   
           1             TelegramBox(x=197.2645966816108, y=87.86136880...   
           1             TelegramBox(x=213.4277122678626, y=98.17841065...   
           1             TelegramBox(x=246.46875, y=102.5, width=1039, ...   
           1             TelegramBox(x=229.29493020241583, y=106.112019...   
           1             TelegramBox(x=186.0526428192383, y=107.9011317...   
           1             TelegramBox(x=210.5003977925535, y=113.3735409...   
           1             TelegramBox(x=215.34472973158375, y=119.929450...   
1959792    1             TelegramBox(x=224.74308855728546, y=30.0541533...   
           2             TelegramBox(x=216.86821575840455, y=55.5473921...   
           2             TelegramBox(x=243.1233821594173, y=63.59903237...   
           2             TelegramBox(x=227.5107482087882, y=64.39426721...   
           2             TelegramBox(x=207.0641823388978, y=236.6764811...   
           3             TelegramBox(x=220.35050350265305, y=538.832990...   
1959793    1             TelegramBox(x=192.34982545934986, y=76.0015834...   
           1             TelegramBox(x=177.49197998854598, y=82.7333777...   
           1             TelegramBox(x=193.23889282118014, y=84.1372306...   
           1             TelegramBox(x=207.77998281387053, y=84.6043379...   
           1             TelegramBox(x=184.45429969872646, y=87.2687331...   
           1   

In [12]:
aggregatedBoxesFrame = allBoxesFrameReindexed.groupby(level=[0, 1]).aggregate({
    'box':
    BoxMatcher.mean
})
display(aggregatedBoxesFrame)

box
subjectKey bestBoxIndex                                                   
1959735    0             TelegramBox(x=183.67013160916906, y=69.2087155...
           1             TelegramBox(x=219.91499669277732, y=90.7737793...
           2             TelegramBox(x=203.4055205555103, y=731.3491153...
1959736    1             TelegramBox(x=199.27109154751565, y=107.603730...
1959737    1             TelegramBox(x=259.66173070723426, y=103.415324...
1959738    1             TelegramBox(x=219.85991187892859, y=65.7198032...
           2             TelegramBox(x=202.93009123957518, y=70.6653239...
           3             TelegramBox(x=219.40498779404675, y=84.2267290...
           4             TelegramBox(x=208.77455986838152, y=122.080924...
           5             TelegramBox(x=200.96875, y=167.0, width=1024.0...
           6             TelegramBox(x=197.90132725772366, y=569.471764...
           7             TelegramBox(x=209.96366001822867, y=584.656763...
1959739    1             TelegramBox(x=241.88075506771008, y=91.4384424...
           2             TelegramBox(x=247.84784399950286, y=1143.20188...
1959740    1             TelegramBox(x=208.582647441238, y=85.929694682...
           2             TelegramBox(x=429.9795057970757, y=232.8091563...
           3             TelegramBox(x=202.67767011376532, y=735.228658...
1959741    1             TelegramBox(x=220.50444591089797, y=172.283812...
           2             TelegramBox(x=1246.1432189730601, y=643.103890...
1959742    1             TelegramBox(x=200.56532798174135, y=80.4700852...
1959743    1             TelegramBox(x=205.79882455579389, y=170.048232...
1959744    1             TelegramBox(x=217.46292706543602, y=88.5402382...
           2             TelegramBox(x=214.87857252218873, y=1261.53575...
1959745    1             TelegramBox(x=237.62469827821815, y=75.6082830...
           2             TelegramBox(x=204.21787987388075, y=81.1823431...
           3             TelegramBox(x=207.03396316746012, y=85.4684206...
           4             TelegramBox(x=188.2113796690944, y=841.0626183...
1959746    1             TelegramBox(x=192.148776983056, y=55.246535500...
           2             TelegramBox(x=202.61015125579502, y=57.4243283...
           3             TelegramBox(x=206.75738747650792, y=101.179760...
...                                                                    ...
1959778    2             TelegramBox(x=223.63147593359008, y=1235.34807...
           3             TelegramBox(x=218.2077823621604, y=1241.833960...
           4             TelegramBox(x=229.33064693263213, y=1246.10470...
1959779    1             TelegramBox(x=186.09750442743723, y=102.086306...
1959780    1             TelegramBox(x=231.35045143525576, y=73.9779102...
1959781    1             TelegramBox(x=197.89610080161935, y=100.292319...
           2             TelegramBox(x=185.5944965979569, y=1205.386396...
1959782    1             TelegramBox(x=222.4975726644932, y=51.48423762...
           2             TelegramBox(x=238.85173397788765, y=70.7087193...
           3             TelegramBox(x=235.57015755879604, y=851.383385...
1959783    1             TelegramBox(x=191.25144048346033, y=96.7135841...
           2             TelegramBox(x=196.4870984607004, y=1165.491185...
1959784    1             TelegramBox(x=240.37603496988527, y=103.923028...
           2             TelegramBox(x=235.0947525036458, y=761.8062917...
1959785    1             TelegramBox(x=215.78630322310823, y=500.936370...
1959786    1             TelegramBox(x=222.43367468050008, y=70.6586345...
1959787    1             TelegramBox(x=177.18931353410653, y=143.866477...
1959788    1             TelegramBox(x=216.25887964038986, y=161.129356...
           2             TelegramBox(x=217.4030007030858, y=773.1672915...
1959789    1             TelegramBox(x=194.4414799892047, y=164.6661413...
           2             TelegramBox(x=191.25528255617076, y=695.141371...
           3         

## Add subject data to aggregated box details

In [13]:
aggregatedBoxesFrame.reset_index(level=[1], drop=False, inplace=True)
aggregatedBoxesFrame = aggregatedBoxesFrame.merge(
    subjectsFrame, how='left', left_index=True, right_index=True)
display(aggregatedBoxesFrame)

,bestBoxIndex,box,collection,huntington_id,ledger,page,telegramNumbers,url,variant
1959735,0,"TelegramBox(x=183.67013160916906, y=69.2087155...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959735,1,"TelegramBox(x=219.91499669277732, y=90.7737793...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959735,2,"TelegramBox(x=203.4055205555103, y=731.3491153...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959736,1,"TelegramBox(x=199.27109154751565, y=107.603730...",EC,mssEC_03_008,3,8,[2],https://panoptes-uploads.zooniverse.org/produc...,
1959737,1,"TelegramBox(x=259.66173070723426, y=103.415324...",EC,mssEC_03_009,3,9,[3],https://panoptes-uploads.zooniverse.org/produc...,
1959738,1,"TelegramBox(x=219.85991187892859, y=65.7198032...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,2,"TelegramBox(x=202.93009123957518, y=70.6653239...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,3,"TelegramBox(x=219.40498779404675, y=84.2267290...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,4,"TelegramBox(x=208.77455986838152, y=122.080924...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,5,"TelegramBox(x=200.96875, y=167.0, width=1024.0...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,


In [14]:
def plotBox(box,
            color=None,
            axis=None,
            showUnreliable=True,
            invertYCoords=True):
    boxFigure = mplplot.gcf()
    boxPlot = axis

    if axis is None:
        boxFigure = mplplot.figure(figsize=(5, 5))
        boxPlot = boxFigure.add_subplot(111, aspect='equal')

    # trivial box reliability criterion
    boxIsReliable = (box.data is None or not isinstance(box.data, dict)
                     or ('nBoxes' in box.data and box.data['nBoxes'] > 1))

    if boxIsReliable or showUnreliable:
        boxPlot.add_patch(
            patches.Rectangle(
                (box.x, box.y),  # (x,y)
                box.width,  # width
                box.height,  # height
                fill=False,
                hatch=None if boxIsReliable else '//',
                alpha=1.0 if boxIsReliable else 0.2,
                ls='-' if boxIsReliable else '--',
                color='r' if color is None else color))
    return boxPlot


def plotBoxes(boxes,
              colors=None,
              axis=None,
              showUnreliable=True,
              invertYCoords=True):
    boxesPlot = axis

    maxYVals = []
    maxXVals = []
    for boxIndex, box in enumerate(boxes):
        boxesPlot = plotBox(
            box,
            color=colors[boxIndex] if colors is not None else 'k',
            axis=boxesPlot,
            showUnreliable=showUnreliable)
        maxXVals.append(box.x + box.width)
        maxYVals.append(box.y + box.height)

    mplplot.xlim(0, np.max(maxXVals))
    mplplot.ylim(0, np.max(maxYVals))

    return boxesPlot

## Quick test: Plot all boxes for the first subject key

In [15]:
for subjectId in np.unique(aggregatedBoxesFrame.index.values)[101:500]:
    print(subjectId, aggregatedBoxesFrame.index.value_counts()[subjectId])
    boxImageFigure = mplplot.figure(figsize=(15, 15))
    idx = pd.IndexSlice
    spectralColorMap = matplotlib.cm.get_cmap('viridis')

    multiBox = aggregatedBoxesFrame.index.value_counts()[subjectId] > 1

    imageUrl = aggregatedBoxesFrame.loc[subjectId].iloc[0][
        'url'] if multiBox else aggregatedBoxesFrame.loc[subjectId]['url']

    imageData = mplplot.imread(urllib.request.urlopen(imageUrl), format='jpeg')
    print(subjectId, type(imageData), imageData.T.shape)
    imageAxes = mplplot.imshow(imageData)

    meanBoxes = aggregatedBoxesFrame.ix[idx[subjectId], idx[:]][['box']]
    maxBoxGroup = len(meanBoxes)
    #boxColors = [ spectralColorMap(boxGroup/float(maxBoxGroup)) for boxGroup, _ in enumerate(meanBoxes.values) ]

    print(type(meanBoxes))
    plotBoxes(
        meanBoxes['box'].values if multiBox else meanBoxes,
        None,
        axis=mplplot.gca(),
        showUnreliable=False,
        invertYCoords=True)

    cur_axes = mplplot.gca()
    cur_axes.axes.get_xaxis().set_ticks([])
    cur_axes.axes.get_yaxis().set_ticks([])
    cur_axes.axes.get_xaxis().set_ticklabels([])
    cur_axes.axes.get_yaxis().set_ticklabels([])

    mplplot.xlim(0, imageData.shape[1])
    mplplot.ylim(0, imageData.shape[0])

    mplplot.tight_layout()
    mplplot.savefig('boxImages/' + str(subjectId) + '_withBoxes.png')
    mplplot.close()
display(aggregatedBoxesFrame)

,bestBoxIndex,box,collection,huntington_id,ledger,page,telegramNumbers,url,variant
1959735,0,"TelegramBox(x=183.67013160916906, y=69.2087155...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959735,1,"TelegramBox(x=219.91499669277732, y=90.7737793...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959735,2,"TelegramBox(x=203.4055205555103, y=731.3491153...",EC,mssEC_03_007,3,7,"[1, 2]",https://panoptes-uploads.zooniverse.org/produc...,
1959736,1,"TelegramBox(x=199.27109154751565, y=107.603730...",EC,mssEC_03_008,3,8,[2],https://panoptes-uploads.zooniverse.org/produc...,
1959737,1,"TelegramBox(x=259.66173070723426, y=103.415324...",EC,mssEC_03_009,3,9,[3],https://panoptes-uploads.zooniverse.org/produc...,
1959738,1,"TelegramBox(x=219.85991187892859, y=65.7198032...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,2,"TelegramBox(x=202.93009123957518, y=70.6653239...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,3,"TelegramBox(x=219.40498779404675, y=84.2267290...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,4,"TelegramBox(x=208.77455986838152, y=122.080924...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,
1959738,5,"TelegramBox(x=200.96875, y=167.0, width=1024.0...",EC,mssEC_03_010,3,10,"[3, 4]",https://panoptes-uploads.zooniverse.org/produc...,


In [16]:
boxFigure = mplplot.figure(figsize=(10, 5))
spectralColorMap = matplotlib.cm.get_cmap('viridis')

allBoxPlot = boxFigure.add_subplot(121, aspect='equal')

allBoxes = allBoxesFrame.ix[idx[1959274], idx[:]][['box', 'bestBoxIndex']]
maxBoxGroup = np.max(allBoxes['bestBoxIndex'].values)
boxColors = [
    spectralColorMap(boxGroup / float(maxBoxGroup))
    for boxGroup in allBoxes['bestBoxIndex'].values
]

plotBoxes(allBoxes['box'].values, boxColors, axis=allBoxPlot)

meanBoxPlot = boxFigure.add_subplot(122, aspect='equal')

meanBoxes = aggregatedBoxesFrame.ix[idx[1959274], idx[:]][['box']]
maxBoxGroup = len(meanBoxes)
boxColors = [
    spectralColorMap(boxGroup / float(maxBoxGroup))
    for boxGroup, _ in enumerate(meanBoxes.values)
]

plotBoxes(meanBoxes['box'].values, boxColors, axis=meanBoxPlot)

NameError: name 'idx' is not defined

## Write aggregated box data to MySQL database

The following commands create a new table for boxes in the `dcwConsensus` database.

```sql
use dcwConsensus;

CREATE TABLE SubjectBoxes (
id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
subjectId INT NOT NULL,
bestBoxIndex INT NOT NULL,
collection VARCHAR(50) NOT NULL DEFAULT 'Unspecified',
ledger INT NOT NULL,
page INT NOT NULL,
meanX DECIMAL(7,3) NOT NULL,
meanY DECIMAL(7,3) NOT NULL,
meanWidth DECIMAL(7,3) NOT NULL,
meanHeight DECIMAL(7,3) NOT NULL,
numBoxesMarked INT NOT NULL
);

CREATE TABLE SubjectTelegrams (
id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
subjectId INT NOT NULL,
collection VARCHAR(50) NOT NULL DEFAULT 'Unspecified',
ledger INT NOT NULL,
page INT NOT NULL,
telegramId INT NOT NULL
);
```

In [17]:
sys.path.append('/Library/Python/2.7/site-packages')
import mysql.connector
#testSubjectData = lineGroupedTranscriptionLineDetails.iloc[0]
'''connection = mysql.connector.connect(user=os.environ['DCW_MYSQL_USER'], password=os.environ['DCW_MYSQL_PASS'],
                              host=os.environ['DCW_MYSQL_HOST'],
                              database=databaseName)'''
connection = mysql.connector.connect(
    user='root',
    password='!Ocus1!Ocus1',
    host='localhost',
    database=databaseName)

cursor = connection.cursor()
sentence = ''
try:
    boxInsertQuery = (
        "INSERT INTO SubjectBoxes (subjectId, bestBoxIndex, collection, ledger, "
        "page, meanX, meanY, meanWidth, meanHeight, numBoxesMarked) "
        "SELECT id, %s, %s, %s, %s, %s, %s, %s, %s, %s "
        "FROM Subjects WHERE huntingtonId = %s")

    telegramInsertQuery = (
        "INSERT INTO SubjectTelegrams (subjectId, collection, ledger, "
        "page, telegramId) "
        "SELECT id, %s, %s, %s, %s "
        "FROM Subjects WHERE huntingtonId = %s")

    currentSubject = -1
    subjectId = None

    # Loop over aggregated lines in consensus data
    for index, row in aggregatedBoxesFrame.iterrows():
        # Insert the aggregated line data
        bestBoxIndex = int(row['bestBoxIndex'])
        boxData = (bestBoxIndex, row['collection'], row['ledger'], row['page'],
                   row['box'].x, row['box'].y, row['box'].width,
                   row['box'].height, row['box'].data['nBoxes'],
                   row['huntington_id'])
        cursor.execute(boxInsertQuery, boxData)

    # Loop over subject frame and insert telegram data
    for index, row in subjectsFrame.iterrows():
        for telegramNumber in row['telegramNumbers']:
            telegramData = (row['collection'], row['ledger'], row['page'],
                            telegramNumber, row['huntington_id'])
            cursor.execute(telegramInsertQuery, telegramData)

except mysql.connector.Error as err:
    print("Failed INSERT: {0}, {1}".format(sentence, err))

connection.commit()

cursor.close()
connection.close()